## Data preprocess

In [10]:
import numpy as np
import pandas as pd


def one_hot_encoding(arr):
    s = set()
    _sd = dict()
    for e in arr:
        s.add(e)

    sl = list(s)
    for i in range(len(sl)):
        encode = np.zeros(len(sl))
        encode[i] = 1
        _sd[sl[i]] = encode

    encoded_data = []
    for k in range(len(arr)):
        encoded_data.append(_sd[arr[k]])

    return np.array(encoded_data)


def data_preprocess(filepath):
    df = pd.read_excel(filepath)
    data = df.to_numpy()

    ip = data[:, 2]

    encoded_ip = one_hot_encoding(ip)

    # print(encoded_ip)

    data[:, 2] = encoded_ip.tolist()

    # print(data)
    expanded_data = []
    for row in data:
        expanded_data.append(np.concatenate([row[:2], row[2], row[3:]]).tolist())

    expanded_data = np.array(expanded_data)

    for i in range(len(expanded_data)):
        for j in range(len(expanded_data[i])):
            expanded_data[i][j] = float(expanded_data[i][j])

    return expanded_data[:290]


filename = "outputv2_5_wait_5s_3(300)"

# Data preprocessing
file_path = f"D:\\model_fit\\training\\training_data\\{filename}.xlsx"
dataset = data_preprocess(file_path)

print(dataset)

[[1.58961e+01 5.43987e+05 1.00000e+00 ... 1.90939e+01 1.86147e+01
  3.10231e+01]
 [5.35940e+00 3.67995e+05 0.00000e+00 ... 1.83406e+01 1.87500e+01
  3.04527e+01]
 [1.67825e+01 8.01369e+05 0.00000e+00 ... 3.82353e+01 3.86018e+01
  8.54700e+00]
 ...
 [2.02960e+00 1.84967e+05 0.00000e+00 ... 1.88854e+01 1.80258e+01
  3.03514e+01]
 [3.31653e+01 2.73870e+05 0.00000e+00 ... 3.87302e+01 3.89831e+01
  2.94671e+01]
 [2.17619e+01 9.61845e+05 0.00000e+00 ... 1.91111e+01 1.91223e+01
  2.96610e+01]]


## XGBOOST

In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X, y = dataset[:, 1:], dataset[:, 0]


train_size = int(len(X) * 0.7)

# 拆分数据集为训练集和测试集

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# time series
# X_train = X[:train_size]
# X_test = X[train_size:]
# y_train = y[:train_size]
# y_test = y[train_size:]

In [12]:
from keras.callbacks import TensorBoard # type: ignore
import os


# log
log_name = "xgboost_v1"
log_dir = os.path.join("logs", "fit", log_name)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

from xgboost import XGBRegressor

# 创建 XGBoost 回归模型
model = XGBRegressor(
    objective="reg:squarederror",  # 损失函数为平方损失函数
    n_estimators=100,  # 迭代次数，即基础学习器的数量
    learning_rate=0.1,  # 学习率，控制每次更新的步长
    max_depth=5,  # 树的最大深度
    min_child_weight=1,  # 叶子节点最小权重
    subsample=0.8,  # 每棵树随机采样的比例
    colsample_bytree=0.8,  # 每棵树随机选择的特征比例
    alpha=0.01,  # L1 正则化项参数
    reg_lambda=0.01,  # L2 正则化项参数
)

# 训练模型
model.fit(X_train, y_train)

XGBRegressor(alpha=0.01, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [13]:
from sklearn.metrics import mean_absolute_error

# 使用模型进行预测
y_pred = model.predict(X_test)

# 评估模型性能
mse = mean_squared_error(y_test, y_pred)
mea = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Squared Error: {mea}")


print(f"    real    |    pred   \n")
for i in range(len(y_test)):
    print(f"    {y_test[i]}     |     {y_pred[i]}      ")


Mean Squared Error: 85.01032464503052
Mean Squared Error: 6.382889672912401
    real    |    pred   

    3.024     |     8.483894348144531      
    13.5913     |     9.331452369689941      
    0.9716     |     6.215897083282471      
    8.7023     |     9.277812957763672      
    6.4232     |     9.998327255249023      
    7.2369     |     11.806024551391602      
    25.7131     |     13.689838409423828      
    3.213     |     5.613520622253418      
    55.6861     |     21.758769989013672      
    29.0796     |     18.862140655517578      
    8.452     |     6.056088447570801      
    0.3898     |     10.098641395568848      
    20.191     |     18.411142349243164      
    21.9265     |     7.241079330444336      
    3.3666     |     7.086533069610596      
    5.7826     |     15.5580415725708      
    16.7158     |     17.99579620361328      
    21.3984     |     7.22969388961792      
    10.1     |     7.317533493041992      
    0.9432     |     5.93335008621215

In [15]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

# 创建 XGBoost 回归模型
model = XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1)

# 使用 GridSearchCV 进行参数调优
param_grid = {
    "max_depth": [3, 5, 7, 10],  # 扩展最大深度的范围
    "learning_rate": [0.01, 0.05, 0.1, 0.3],  # 扩展学习率的范围
    "n_estimators": [100, 200, 300, 500],  # 扩展树的数量范围
    "gamma": [0, 0.1, 0.2, 0.3],  # 扩展 gamma 参数范围
    "subsample": [0.7, 0.8, 0.9, 1.0],  # 扩展子样本比例范围
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],  # 扩展列采样比例范围
    "reg_alpha": [0, 0.01, 0.1, 1.0],  # 扩展 L1 正则化参数范围
    "reg_lambda": [0, 0.01, 0.1, 1.0, 10.0],  # 扩展 L2 正则化参数范围
}


grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error"
)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)

# 使用最佳参数重新训练模型
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)


# 预测
preds = best_model.predict(X_test)

# 评估模型
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE: {rmse}")

# 计算 MAE
mae = mean_absolute_error(y_test, preds)
print(f"MAE: {mae}")


print(f"    real    |    pred   \n")
for i in range(len(y_test)):
    print(f"    {y_test[i]}     |     {y_pred[i]}      ")

KeyboardInterrupt: 